In [ ]:
###client.delete_collection('yt_demo')
import os
# os.environ['BASE-PROJECT-DIR'] = '/home/adeilson/llm-study'
# os.environ['DATA-PROJECT-DIR'] = os.environ.get('BASE-PROJECT-DIR') + '/datasets'
# os.environ["PYTHONPATH"] = os.environ.get('BASE-PROJECT-DIR') + '/commons'
os.environ.get("PYTHONPATH")

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

import load_datasets
from pprint import pprint

#client = chromadb.Client()
#sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

PERSISTED_CHROMADB_DIR = '/home/adeilson/llm-study/persisted_chromadb'
client = chromadb.PersistentClient(path=PERSISTED_CHROMADB_DIR)

In [1]:
#collection = client.get_or_create_collection(name="yt_demo", embedding_function=sentence_transformer_ef)

import chromadb
client = chromadb.Client()
collection = client.get_or_create_collection(name="yt_demo")
collection.add(
    #embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]],
    documents=["This is a document about cat", "This is a document about car"],
    metadatas=[{"category": "animal"}, {"category": "vehicle"}],
    ids=["1", "2"]
)

In [ ]:
collection.count()

In [11]:
results = collection.query(
    query_texts=["taxi"],
    n_results=2
)
results['documents'][0]


['This is a document about car', 'This is a document about cat']

In [12]:
len(results['documents'][0])

2

In [ ]:
file_data = load_datasets.read_all_txt_files()

documents = []
metadatas = []
ids = []

for index, data in enumerate(file_data):
    documents.append(data['content'])
    metadatas.append({'source': data['file_name']})
    ids.append(str(index + 1))

txt_collection = client.create_collection(
    name="txt_collection",
    metadata={"hnsw:space": "cosine"} # l2 is the default
)

txt_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
txt_collection.count()
#txt_collection.metadata

In [ ]:
txt_collection = client.get_collection("txt_collection")

### Querying

In [ ]:
results = txt_collection.query(
    query_texts=["Who has experience as CTO and experience in AWS?"],
    n_results=1,
    include=["metadatas","documents","distances"]
)
pprint(results)


In [ ]:
results = txt_collection.query(
    query_texts=["Who knows TD security and Selenium?"],
    n_results=1
)
pprint(results)

In [ ]:
results = txt_collection.query(
    query_texts=["proteção de dados?"],
    n_results=2
)
pprint(results)

### Filtering

In [ ]:
results = txt_collection.query(
    query_texts=["Knows how to architect to AWS"],
    n_results=1
    #, where_document={"$contains":"data engineering"}
    , where={"source": "adeilson-cv.txt"}
)
pprint(results)

### Embedding

ChromaDB uses the Sentence Transformers **all-MiniLM-L6-v2** model by default.  
You can use any other model for creating embeddings.  

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

documents = []
embeddings = []
metadatas = []
ids = []

for index, data in enumerate(file_data):
    documents.append(data['content'])
    embedding = model.encode(data['content']).tolist()
    embeddings.append(embedding)
    metadatas.append({'source': data['file_name']})
    ids.append(str(index + 1))


In [ ]:
#if check_if_collection_exists("txt_collection_emb"):
# client.delete_collection("txt_collection_emb")

txt_collection_emb = client.get_or_create_collection("txt_collection_emb")

txt_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)


In [ ]:
#### This query is not working well
query = "Who has experience as CTO and experience in AWS?"
input_em = model.encode(query).tolist()

results = txt_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
pprint(results)
